In [1]:
!pip install pygame
!pip install gtts
!pip install yt-dlp
!pip install requests
!pip install --upgrade requests
!pip install pydub
!apt-get install ffmpeg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.9/171.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 46.2 MB/s eta 0:00:00
Extracting cookies from chrome
ERROR: could not find chrome cookies database in "/root/.config/google-chrome"
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.6 MB/s eta 0:00:00


In [4]:
import yt_dlp
import requests
import os
from gtts import gTTS

DEEPGRAM_API_KEY = ''  # Your Deepgram API key
DEEPGRAM_API_URL = ''  # Ensure this is the correct API endpoint

def clear_old_audio_files():
    """Remove old audio files."""
    for file in os.listdir('.'):
        if file.endswith('.mp3'):
            os.remove(file)
            print(f"Removed old audio file: {file}")

def download_video(video_url):
    clear_old_audio_files()  # Clear previous audio files

    options = {
        'format': 'bestvideo+bestaudio/best',
        'outtmpl': '%(title)s.%(ext)s',
        'merge_output_format': 'mp4',
        'progress_hooks': [hook],
        'noplaylist': True
    }

    with yt_dlp.YoutubeDL(options) as ydl:
        try:
            ydl.download([video_url])
            print("Download completed successfully.")
            audio_file = extract_audio(video_url)  # Extract audio from the downloaded video
            if audio_file:
                transcription = transcribe_audio(audio_file)  # Use the extracted audio file
                if transcription:
                    generate_speech(transcription)  # Generate speech using gTTS
            else:
                print("No audio file found after extraction.")
        except Exception as e:
            print(f"An error occurred: {e}")

def extract_audio(video_url):
    audio_options = {
        'format': 'bestaudio/best',
        'outtmpl': '%(title)s.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'progress_hooks': [hook],
    }

    with yt_dlp.YoutubeDL(audio_options) as ydl:
        try:
            ydl.download([video_url])
            print("Audio extraction completed successfully.")
            current_files = os.listdir('.')
            audio_files = [f for f in current_files if f.endswith('.mp3')]
            if audio_files:
                return audio_files[0]  # Return the first found audio file
            else:
                print("No audio files found after extraction.")
                return None
        except Exception as e:
            print(f"An error occurred during audio extraction: {e}")
            return None

def hook(d):
    if d['status'] == 'finished':
        print(f"\nDone downloading: {d['filename']}")

def transcribe_audio(audio_file):
    headers = {
        'Authorization': f'Token {DEEPGRAM_API_KEY}',
        'Content-Type': 'audio/mp3'
    }

    try:
        session = requests.Session()

        with open(audio_file, 'rb') as f:
            response = session.post(DEEPGRAM_API_URL, headers=headers, data=f)

        response.raise_for_status()

        transcription = response.json()
        print("Transcription completed successfully:")
        transcript = transcription['results']['channels'][0]['alternatives'][0]['transcript']
        print(transcript)
        return transcript

    except requests.exceptions.RequestException as e:
        print(f"An error occurred during transcription: {e}")
        if e.response:
            print(f"Response status code: {e.response.status_code}")
            print(f"Response text: {e.response.text}")
        else:
            print("No response received from the server.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

    return None

def generate_speech(text):
    try:
        tts = gTTS(text=text, lang='en')
        tts.save('generated_audio.mp3')
        print("Audio generated successfully and saved as 'generated_audio.mp3'.")
    except Exception as e:
        print(f"An error occurred during speech generation: {e}")

# Example usage
video_url = 'https://youtu.be/Vds8ddYXYZY?si=5sv8I3DrzxQBDfYq'  # Replace with the desired YouTube video URL
download_video(video_url)


[youtube] Extracting URL: https://youtu.be/Vds8ddYXYZY?si=5sv8I3DrzxQBDfYq
[youtube] Vds8ddYXYZY: Downloading webpage
[youtube] Vds8ddYXYZY: Downloading tv client config
[youtube] Vds8ddYXYZY: Downloading player f6e09c70
[youtube] Vds8ddYXYZY: Downloading tv player API JSON
[youtube] Vds8ddYXYZY: Downloading ios player API JSON
[youtube] Vds8ddYXYZY: Downloading m3u8 information
[info] Vds8ddYXYZY: Downloading 1 format(s): 616+251
[download] Ed Sheeran - Shape Of You (Lyrics).mp4 has already been downloaded
Download completed successfully.
[youtube] Extracting URL: https://youtu.be/Vds8ddYXYZY?si=5sv8I3DrzxQBDfYq
[youtube] Vds8ddYXYZY: Downloading webpage
[youtube] Vds8ddYXYZY: Downloading tv client config
[youtube] Vds8ddYXYZY: Downloading player f6e09c70
[youtube] Vds8ddYXYZY: Downloading tv player API JSON
[youtube] Vds8ddYXYZY: Downloading ios player API JSON
[youtube] Vds8ddYXYZY: Downloading m3u8 information
[info] Vds8ddYXYZY: Downloading 1 format(s): 251
[download] Destination: